<a href="https://colab.research.google.com/github/aksingh21/Probabilistic-Programming-PyMC3/blob/master/PyMc3NewModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Conditional Autoregressive (CAR)**

---


Tutorial from https://docs.pymc.io/notebooks/PyMC3_tips_and_heuristic.html

In [1]:
%pylab inline

import scipy.stats as stats
import pymc3 as pm
from theano import shared
import theano
import theano.tensor as tt

floarX = "float32"

%config InlineBackend.fegure_format = 'retina'
plt.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


**Data**

In [0]:
county = np.array(["skye_lochalsh", "banff_buchan", "caithness,berwickshire", "ross_cromarty",
                   "orkney", "moray", "shetland", "lochaber", "gordon", "western_isles",
                   "sutherland", "nairn", "wigtown", "NE.fife", "kincardine", "badenoch",
                   "ettrick", "inverness", "roxburgh", "angus", "aberdeen", "argyll_bute",
                   "clydesdale", "kirkcaldy", "dunfermline", "nithsdale", "east_lothian",
                   "perth_kinross", "west_lothian", "cumnock_doon", "stewartry", "midlothian",
                   "stirling", "kyle_carrick", "inverclyde", "cunninghame", "monklands",
                   "dumbarton", "clydebank", "renfrew", "falkirk", "clackmannan", "motherwell",
                   "edinburgh", "kilmarnock", "east_kilbride", "hamilton", "glasgow", "dundee",
                   "cumbernauld", "bearsden", "eastwood", "strathkelvin", "tweeddale",
                   "annandale"])

#observed
O = np.array([9, 39, 11, 9, 15, 8, 26, 7, 6, 20, 13, 5, 3, 8, 17, 9, 2, 7, 9, 7, 16,
              31, 11, 7, 19, 15, 7, 10, 16, 11, 5, 3, 7, 8, 11, 9, 11, 8, 6, 4, 10,
              8, 2, 6, 19, 3, 2, 3, 28, 6, 1, 1, 1, 1, 0, 0])
N = len(O)

#expected (E) rates, based on the age of the local population
E = np.array([1.4, 8.7, 3.0, 2.5, 4.3, 2.4, 8.1, 2.3, 2.0, 6.6, 4.4, 1.8, 1.1, 3.3,
              7.8, 4.6, 1.1, 4.2, 5.5, 4.4, 10.5, 22.7, 8.8, 5.6, 15.5, 12.5, 6.0,
              9.0, 14.4, 10.2, 4.8, 2.9, 7.0, 8.5, 12.3, 10.1, 12.7, 9.4, 7.2, 5.3,
              18.8, 15.8, 4.3, 14.6, 50.7, 8.2, 5.6, 9.3, 88.7, 19.6, 3.4, 3.6, 5.7,
              7.0, 4.2, 1.8])

logE = np.log(E)

# proportion of the population engaged in agriculture, forestry, or fishing (AFF)
aff = np.array([16, 16, 10, 24, 10, 24, 10, 7, 7, 16, 7, 16, 10, 24, 7, 16, 10, 7,
                7, 10, 7, 16, 10, 7, 1, 1, 7, 7, 10, 10, 7, 24, 10, 7, 7, 0, 10, 1,
                16, 0, 1, 16, 16, 0, 1, 7, 1, 1, 0, 1, 1, 0, 1, 1, 16, 10])/10.


#Spatial adjacency information
adj = np.array([[5, 9, 11,19],
                [7, 10],
                [6, 12],
                [18,20,28],
                [1, 11,12,13,19],
                [3, 8],
                [2, 10,13,16,17],
                [6],
                [1, 11,17,19,23,29],
                [2, 7, 16,22],
                [1, 5, 9, 12],
                [3, 5, 11],
                [5, 7, 17,19],
                [31,32,35],
                [25,29,50],
                [7, 10,17,21,22,29],
                [7, 9, 13,16,19,29],
                [4,20, 28,33,55,56],
                [1, 5, 9, 13,17],
                [4, 18,55],
                [16,29,50],
                [10,16],
                [9, 29,34,36,37,39],
                [27,30,31,44,47,48,55,56],
                [15,26,29],
                [25,29,42,43],
                [24,31,32,55],
                [4, 18,33,45],
                [9, 15,16,17,21,23,25,26,34,43,50],
                [24,38,42,44,45,56],
                [14,24,27,32,35,46,47],
                [14,27,31,35],
                [18,28,45,56],
                [23,29,39,40,42,43,51,52,54],
                [14,31,32,37,46],
                [23,37,39,41],
                [23,35,36,41,46],
                [30,42,44,49,51,54],
                [23,34,36,40,41],
                [34,39,41,49,52],
                [36,37,39,40,46,49,53],
                [26,30,34,38,43,51],
                [26,29,34,42],
                [24,30,38,48,49],
                [28,30,33,56],
                [31,35,37,41,47,53],
                [24,31,46,48,49,53],
                [24,44,47,49],
                [38,40,41,44,47,48,52,53,54],
                [15,21,29],
                [34,38,42,54],
                [34,40,49,54],
                [41,46,47,49],
                [34,38,49,51,52],
                [18,20,24,27,56],
                [18,24,30,33,45,55]])

#change to Python indexing (i.e. -1)
for i in range (len(adj)):
  for j in range (len(adj[i])):
    adj[i][j] = adj[i][j]-1
    

# spatial weight
weights = np.array([[1,1,1,1],
                    [1,1],
                    [1,1],
                    [1,1,1],
                    [1,1,1,1,1],
                    [1,1],
                    [1,1,1,1,1],
                    [1],
                    [1,1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1],
                    [1,1,1,1],
                    [1,1,1],
                    [1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1],
                    [1,1,1],
                    [1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1,1,1,1],
                    [1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1,1,1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1,1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1,1,1,1,1],
                    [1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1,1],
                    [1,1,1,1,1,1]])

Wplus = np.asarray([sum(w) for w in weights])

**PyMC3 implementation using theano.scan**

In [33]:
value = np.asarray(np.random.randn(N,), dtype = theano.config.floatX)

maxwz = max([sum(w) for w in weights])

N = len(weights)
wmat = np.zeros((N,maxz))
amat = np.zeros((N, maxwz), dtype='int32')

for i, w in enumerate(weights):
  wmat[i, np.arange(len(w))] = w
  amat[i, np.arange(len(w))] = adj[i]
  
#defining a tensor variables
x = tt.vector("x")
x.tag.test_value = value
w = tt.matrix("w")

#provide Theano with a default
w.tag.test_value = wmat
a = tt.matrix("a", dtype='int32')
a.tag.test_value = amat


def get_mu(w, a):
  a1 = tt.cast(a, 'int32')
  return tt.sum(w*x[a1])/tt.sum(w)

results, _ = theano.scan(fn=get_mu, sequences=[w,a])
compute_elementwise = theano.function(inputs=[x, w, a], outputs=results)

print(compute_elementwise(value,wmat,amat))


def mu_phi(value):
  N = len(weights)
  #calculate mu based on average neighbours
  mu = np.array([np.sum(weights[i]*value[adj[i]])/Wplus[i] for i in range(N)])
  return mu

print(mu_phi(value))

[ 3.76698858e-01 -2.66953831e-01 -6.93575078e-01  2.20222393e-01
 -3.98220820e-01 -9.54106273e-01 -3.14377624e-01 -9.24108500e-01
  2.74775646e-01  2.33189208e-01 -7.80516034e-02  8.90493663e-01
  1.15997163e+00 -2.57908474e-01  2.54432223e-01  3.23376239e-01
  1.92167477e-01  5.33818656e-01  6.29965874e-02 -4.26398362e-01
  7.73724717e-01 -6.42745614e-01  3.03850823e-02  1.40758729e-01
  6.94557585e-01 -3.68043671e-01 -2.31507322e-01  1.39991601e-01
 -1.09721925e-02  1.13164214e-01 -1.51086458e-01 -3.01623519e-01
 -1.53630219e-01 -2.00215654e-01 -9.69223760e-02  1.12697140e-03
  3.35129751e-01  1.51270176e-01  2.84839713e-02 -3.07812804e-01
  4.96358589e-01 -7.19215589e-01  1.95862925e-01 -6.00526381e-01
  4.87753163e-02  3.34043105e-01  3.94796868e-01  8.08646904e-01
 -9.28190617e-03  1.00612215e+00 -6.61931040e-01 -4.74539358e-02
  5.29165131e-01 -5.97746957e-01 -4.47888934e-03 -6.12384901e-02]
[ 3.76698858e-01 -2.66953831e-01 -6.93575078e-01  2.20222393e-01
 -3.98220820e-01 -9.5410

In [0]:
from theano import scan
floatX = "float32"

from pymc3.distributions import continuous
from pymc3.distributions import distribution


In [27]:
class CAR(distribution.Continuous):
  """
  Conditional Autoregressive (CAR) distribution
  
  Parameters
  __________
  a : list of adjacency information
  w : list of weight information
  tau : precision at each location
  """

array([0, 1])